In [1]:
!uv add boto3

Resolved 165 packages in 3ms
Audited 143 packages in 0.15ms


In [2]:
import json
import boto3

In [4]:
region = 'eu-west-1' # "us-east-1"

bedrock_boto_client = boto3.client("bedrock-runtime", region_name=region)

In [54]:
# model_id = "minimax.minimax-m2"
model_id = "openai.gpt-oss-120b-1:0"

In [57]:
user_prompt = "Describe the purpose of a 'hello world' program in one line."

messages = [
    {"role": "user", "content": user_prompt}
]

request = json.dumps({
    "messages": messages,
    "max_tokens": 512,
})

In [58]:
response = bedrock_boto_client.invoke_model(
    modelId=model_id,
    body=request
)

model_response = json.loads(response["body"].read())
model_response

{'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': '<reasoning>The user asks: "Describe the purpose of a \'hello world\' program in one line." So we need to give a one-line description. Provide concise answer.\n\nProbably: "A \'Hello, World!\' program demonstrates the basic syntax and verifies that a language\'s toolchain successfully compiles and runs code." Something like that. One line.\n\nI\'ll output a single sentence.\n\n</reasoning>A “Hello, World!” program simply verifies that a language’s compiler/interpreter and runtime are correctly set up by printing a basic output.',
    'refusal': None,
    'role': 'assistant'}}],
 'created': 1769767687,
 'id': 'chatcmpl-33510a6a-0aa6-4b75-a8cb-122d142929f0',
 'model': 'openai.gpt-oss-120b-1:0',
 'object': 'chat.completion',
 'service_tier': 'default',
 'usage': {'completion_tokens': 113, 'prompt_tokens': 79, 'total_tokens': 192}}

In [59]:
output_text = model_response['choices'][0]['message']['content']
print(output_text)

<reasoning>The user asks: "Describe the purpose of a 'hello world' program in one line." So we need to give a one-line description. Provide concise answer.

Probably: "A 'Hello, World!' program demonstrates the basic syntax and verifies that a language's toolchain successfully compiles and runs code." Something like that. One line.

I'll output a single sentence.

</reasoning>A “Hello, World!” program simply verifies that a language’s compiler/interpreter and runtime are correctly set up by printing a basic output.


In [61]:
model_id = 'mistral.mistral-large-2402-v1:0'

user_prompt = "Describe the purpose of a 'hello world' program in one line."

# also works
# messages = [
#     {"role": "user", "content": user_prompt}
# ]

messages = [
    {"role": "user", "content":[{"text": user_prompt}]}
]

request = json.dumps({
    "messages": messages,
    "max_tokens": 512,
})

response = bedrock_boto_client.invoke_model(
    modelId=model_id,
    body=request
)

model_response = json.loads(response["body"].read())
model_response

{'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "The purpose of a 'hello world' program is to provide a simple introduction to a programming language, demonstrating its basic syntax and confirming that the environment is set up correctly."},
   'stop_reason': 'stop'}]}

In [62]:
output_text = model_response['choices'][0]['message']['content']
print(output_text)

The purpose of a 'hello world' program is to provide a simple introduction to a programming language, demonstrating its basic syntax and confirming that the environment is set up correctly.


In [68]:
model_id = 'global.amazon.nova-2-lite-v1:0'

user_prompt = "Describe the purpose of a 'hello world' program in one line."

messages = [
    {"role": "user", "content":[{"text": user_prompt}]}
]

request = json.dumps({
    "messages": messages,
})

response = bedrock_boto_client.invoke_model(
    modelId=model_id,
    body=request
)

model_response = json.loads(response["body"].read())
model_response

{'output': {'message': {'content': [{'text': "A 'hello world' program serves as a simple introductory example to demonstrate the basic syntax and execution flow of a programming language."}],
   'role': 'assistant'}},
 'stopReason': 'end_turn',
 'usage': {'inputTokens': 60,
  'outputTokens': 28,
  'totalTokens': 88,
  'cacheReadInputTokenCount': 0,
  'cacheWriteInputTokenCount': 0}}

In [69]:
output_text = model_response['output']['message']['content'][0]['text']
print(output_text)

A 'hello world' program serves as a simple introductory example to demonstrate the basic syntax and execution flow of a programming language.


In [80]:
model_id = 'global.anthropic.claude-sonnet-4-5-20250929-v1:0'

user_prompt = "Describe the purpose of a 'hello world' program in one line."

messages = [
    {"role": "user", "content":[{"type": "text", "text": user_prompt}]}
]

request = json.dumps({
    "messages": messages,
    "max_tokens": 1024,
    "anthropic_version": "bedrock-2023-05-31"
})

response = bedrock_boto_client.invoke_model(
    modelId=model_id,
    body=request
)

model_response = json.loads(response["body"].read())
model_response

{'model': 'claude-sonnet-4-5-20250929',
 'id': 'msg_bdrk_017gYXDQa1Gho8ZrnUaHpnnd',
 'type': 'message',
 'role': 'assistant',
 'content': [{'type': 'text',
   'text': "A 'Hello World' program demonstrates the basic syntax of a programming language and verifies that the development environment is properly configured."}],
 'stop_reason': 'end_turn',
 'stop_sequence': None,
 'usage': {'input_tokens': 23,
  'cache_creation_input_tokens': 0,
  'cache_read_input_tokens': 0,
  'cache_creation': {'ephemeral_5m_input_tokens': 0,
   'ephemeral_1h_input_tokens': 0},
  'output_tokens': 29}}

In [81]:
output_text = model_response['content'][0]['text']
print(output_text)

A 'Hello World' program demonstrates the basic syntax of a programming language and verifies that the development environment is properly configured.


In [83]:
model_id = 'global.amazon.nova-2-lite-v1:0'

user_prompt = "Describe the purpose of a 'hello world' program in one line."

messages = [
    {"role": "user", "content":[{"text": user_prompt}]}
]

request = json.dumps({
    "messages": messages,
})

response = bedrock_boto_client.invoke_model_with_response_stream(
    modelId=model_id,
    body=request
)

stream = response['body']

for event in stream:
    chunk = json.loads(event['chunk']['bytes'])
    if 'contentBlockDelta' not in chunk:
        continue
    content_block  = chunk['contentBlockDelta']
    if 'delta' not in content_block:
        continue
    print(content_block['delta']['text'], end='')

A 'hello world' program serves as a simple introductory example to demonstrate the basic syntax and execution flow of a programming language.

In [87]:
model_id = 'global.anthropic.claude-sonnet-4-5-20250929-v1:0'

user_prompt = "Describe the purpose of a 'hello world' program in one line."

messages = [
    {"role": "user", "content":[{"type": "text", "text": user_prompt}]}
]

request = json.dumps({
    "messages": messages,
    "max_tokens": 1024,
    "anthropic_version": "bedrock-2023-05-31"
})

response = bedrock_boto_client.invoke_model_with_response_stream(
    modelId=model_id,
    body=request
)

stream = response['body']

for event in stream:
    chunk = json.loads(event['chunk']['bytes'])
    # print(chunk)

    if 'delta' not in chunk:
        continue
    delta = chunk['delta']
    if 'text' not in delta:
        continue
    print(delta['text'], end='')

A "Hello World" program demonstrates the basic syntax of a programming language by displaying a simple message to confirm the development environment is working correctly.

In [89]:
from dotenv import load_dotenv
load_dotenv()

True

In [90]:
import os
from openai import OpenAI

base_url = f"https://bedrock-runtime.{region}.amazonaws.com/openai/v1"

bedrock_openai_client = OpenAI(
    base_url=base_url,
    api_key=os.getenv("AWS_BEARER_TOKEN_BEDROCK")
)

In [103]:
# model_id = "minimax.minimax-m2"
model_id = "openai.gpt-oss-120b-1:0"
# model_id = 'mistral.mistral-large-2402-v1:0'  # not supported

In [104]:
user_prompt = "Describe the purpose of a 'hello world' program in one line."

messages = [
    {"role": "user", "content": user_prompt}
]

response = bedrock_openai_client.chat.completions.create(
    model=model_id,
    messages=messages,
    max_tokens=1000,
)

In [105]:
response.choices[0].message.content

'<reasoning>The user asks: "Describe the purpose of a \'hello world\' program in one line." So we need a one-line description. Probably something like: "It shows that a language\'s compiler/interpreter works and demonstrates basic syntax." Provide in one line. So answer a single line.</reasoning>A “Hello, World!” program simply verifies that a language’s toolchain is set up correctly and demonstrates the most basic syntax for outputting text.'

In [106]:
from anthropic import AnthropicBedrock

In [111]:
anthropic_bedrock_client = AnthropicBedrock(
    aws_region=region,
)

In [112]:
model_id = 'global.anthropic.claude-sonnet-4-5-20250929-v1:0'

messages = [
    {"role": "user", "content": "Hello"}
]

instructions = "You are a helpful assistant. Reply with emojis."

message = anthropic_bedrock_client.messages.create(
    model=model_id,
    max_tokens=1024,
    system=instructions,
    messages=messages
)

print(message.content[0].text)

Hello! 👋😊 How can I help you today? ✨


In [113]:
messages = [
    {"role": "user", "content": "Tell me a bedtime story (100 words)"}
]

with anthropic_bedrock_client.messages.stream(
    max_tokens=1024,
    messages=messages,
    model=model_id,
) as stream:
  for text in stream.text_stream:
      print(text, end="", flush=True)

# The Sleepy Star

High above, a tiny star named Luna grew drowsy. She'd been shining all night and her light began to flicker.

"Time for bed," whispered the Moon kindly.

"But who will light the way?" Luna worried.

"Look," said the Moon.

Below, the Sun was rising, painting the sky pink and gold. 

"We take turns, little one. Now the Sun will watch over Earth while you rest."

Luna yawned, her light growing soft and golden. As she drifted behind the clouds, she smiled. Tomorrow night, she'd shine brightly again.

And she dreamed of dawn.

*Goodnight.* ✨

In [114]:
from pydantic import BaseModel

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

In [116]:
structured_output_tool = {
    "name": "final_result",
    "description": "Tool for structured output",
    "input_schema": CalendarEvent.model_json_schema()
}

In [118]:
instructions = "Extract the event information."
content = "Alice and Bob are going to a science fair on Friday."

messages = [{"role": "user", "content": content}]

response = anthropic_bedrock_client.messages.create(
    model=model_id,
    max_tokens=1024,
    messages=messages,
    tools=[structured_output_tool],
    tool_choice={"type": "tool", "name": structured_output_tool['name']}
)

In [123]:
event = CalendarEvent.model_validate(response.content[0].input)
print(event)

name='Science Fair' date='Friday' participants=['Alice', 'Bob']


In [127]:
def llm_structured(instructions, user_prompt, output_type, model_name):
    structured_output_tool = {
        "name": "final_result",
        "description": "Tool for structured output",
        "input_schema": output_type.model_json_schema()
    }

    messages = [{"role": "user", "content": user_prompt}]

    response = anthropic_bedrock_client.messages.create(
        model=model_name,
        max_tokens=1024,
        system=instructions,
        messages=messages,
        tools=[structured_output_tool],
        tool_choice={"type": "tool", "name": structured_output_tool['name']}
    )

    raw_input = response.content[0].input
    # print(raw_input)
    output = output_type.model_validate(raw_input)
    return output

In [128]:
llm_structured(instructions, content, CalendarEvent, model_id)

CalendarEvent(name='science fair', date='Friday', participants=['Alice', 'Bob'])

In [129]:
import rag
index = rag.initialize_index()

Indexed 385 chunks from 95 documents


['answer',
 'found_answer',
 'confidence',
 'confidence_explanation',
 'answer_type',
 'followup_questions']

In [142]:
class AnthropicBedrockRAG(rag.RAG):

    def llm(self, user_prompt):
        instructions = f"""
            {self.rag_instructions}

            CRITICAL: make sure to create all these fields
            in our output: {self.output_type.model_json_schema()['required']}

            If we don't produce them, we fail our validation and we will have to 
            make another request. 

            for setting followup_questions, analyze the results and think of 
            what you suggest for the user to ask next
        """
        
        return llm_structured(
            instructions,
            user_prompt,
            self.output_type,
            self.model_name
        )
        

In [143]:
bedrock_rag = AnthropicBedrockRAG(
    index,
    anthropic_bedrock_client,
    model_name=model_id
)

response = bedrock_rag.rag("llm as a judge")
print(response.answer[:50] + "...")

ValidationError: 3 validation errors for RAGResponse
confidence_explanation
  Field required [type=missing, input_value={'answer': '# LLM as a Ju...rue, 'confidence': 0.95}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
answer_type
  Field required [type=missing, input_value={'answer': '# LLM as a Ju...rue, 'confidence': 0.95}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
followup_questions
  Field required [type=missing, input_value={'answer': '# LLM as a Ju...rue, 'confidence': 0.95}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing